### EDA IOWA Dataset - Filter the dataset to have full years til end 2023

**Status:** PUBLIC Distribution <br>

**Author:** Jaume Manero IE<br>
**Date created:** 2021/02/1<br>
**Last modified:** 2024/01/30<br>
**Description:** Analysis of IOWA Dataset

This dataset contains every wholesale purchase of liquor in the State of Iowa by retailers for sale to individuals since January 1, 2012 til 2023
The State of Iowa controls the wholesale distribution of liquor intended for retail sale, which means this dataset offers a complete view of retail liquor sales in the entire state. The dataset contains every wholesale order of liquor by all grocery stores, liquor stores, convenience stores, etc., with details about the store and location, the exact liquor brand and size, and the number of bottles ordered.

In [1]:
# Liquor Sales : 
#    file: https://mydata.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy
# US County Boundaries & FIP Codes
#    file: https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_20m.zip
# Counties Population
#    file: https://data.iowa.gov/Community-Demographics/County-Population-in-Iowa-by-Year/qtnr-zsrc
# Cities in IOWA population
#    file: https://data.iowa.gov/Community-Demographics/Total-City-Population-by-Year/acem-thbp

In [1]:
import pandas as pd
from thefuzz import fuzz
import folium
import warnings
import geopandas
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.figure_factory as ff
import plotly.graph_objects as go
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
file = 'Iowa_Liquor_Sales_20240319.csv'
df = pd.read_csv(file, header=0)

In [3]:
# let's see the years in the dataset
# first we create a datetime column
df['date_datetime'] = pd.to_datetime(df['Date'])
# Sort file by datetime
df = df.sort_values(by='date_datetime')
# Let's see the unique years
print(df['date_datetime'].dt.year.unique())
# 

[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]


In [4]:
df.describe()

,Store Number,County Number,Category,Vendor Number,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),date_datetime
count,2.859269e+07,2.413284e+07,2.857571e+07,2.859268e+07,2.859269e+07,2.859269e+07,2.859268e+07,2.859268e+07,2.859269e+07,2.859268e+07,2.859269e+07,2.859269e+07,28592688
mean,3.893087e+03,5.725913e+01,1.050259e+06,2.725597e+02,1.214036e+01,8.791242e+02,1.064285e+01,1.597455e+01,1.079254e+01,1.445597e+02,9.147597e+00,2.413841e+00,2018-05-25 08:37:02.129630976
min,2.106000e+03,1.000000e+00,1.012200e+05,1.000000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.680000e+02,-9.720000e+03,-1.344000e+03,-3.550400e+02,2012-01-03 00:00:00
25%,2.621000e+03,3.100000e+01,1.012200e+06,1.250000e+02,6.000000e+00,7.500000e+02,5.650000e+00,8.480000e+00,3.000000e+00,3.496000e+01,1.500000e+00,4.000000e-01,2015-06-09 00:00:00
50%,3.886000e+03,6.200000e+01,1.031200e+06,2.600000e+02,1.200000e+01,7.500000e+02,8.490000e+00,1.274000e+01,6.000000e+00,7.728000e+01,4.800000e+00,1.260000e+00,2018-07-25 00:00:00
75%,4.794000e+03,7.700000e+01,1.062400e+06,3.950000e+02,1.200000e+01,1.000000e+03,1.299000e+01,1.949000e+01,1.200000e+01,1.485600e+02,1.050000e+01,2.770000e+00,2021-06-05 00:00:00
max,1.037000e+04,9.900000e+01,1.901200e+06,9.870000e+02,3.360000e+02,3.780000e+05,2.498902e+04,3.748353e+04,1.500000e+04,2.795573e+05,1.500000e+04,3.962580e+03,2024-02-29 00:00:00
std,1.268075e+03,2.728710e+01,8.303833e+04,1.448834e+02,7.760099e+00,6.264891e+02,1.304729e+01,1.956967e+01,3.037771e+01,5.094256e+02,3.615571e+01,9.551434e+00,NaN


In [5]:
df.tail(10)

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),date_datetime
4174989,INV-67803000005,02/29/2024,5859,LIQUOR TOBACCO & GROCERY - MASON CITY,18 NORTH MONROE AVENUE,MASON CITY,50401,POINT (-93.209221975 43.152355982),NaN,CERRO GORDO,...,FIVE STAR,24,375,2.09,3.14,48,150.72,18.00,4.75,2024-02-29
4164568,INV-67791500076,02/29/2024,4829,CENTRAL CITY 2,1501 MICHIGAN AVE,DES MOINES,50314,POINT (-93.613286034 41.605834999),NaN,POLK,...,SMIRNOFF KISSED CARAMEL,12,750,8.25,12.38,24,297.12,18.00,4.75,2024-02-29
4174986,INV-67799600035,02/29/2024,6129,EAST END LIQUOR / DES MOINES,3804 HUBBELL AVE,DES MOINES,50317,POINT (-93.541625959 41.631082004),NaN,POLK,...,STATE VODKA,12,750,3.50,5.25,12,63.00,9.00,2.37,2024-02-29
4164562,INV-67792200014,02/29/2024,2106,HILLSTREET NEWS AND TOBACCO,2217 COLLEGE,CEDAR FALLS,50613,POINT (-92.456175029 42.517074986),NaN,BLACK HAWK,...,TEMPLETON RYE 4YR,6,750,15.40,23.10,6,138.60,4.50,1.18,2024-02-29
4174983,INV-67797100009,02/29/2024,10057,KWIK STAR #1186 / ALTOONA,2030 21ST ST NW,ALTOONA,50009,POINT (-93.48478601 41.670812979),NaN,POLK,...,CEDAR RIDGE MALTED RYE,6,750,19.67,29.51,6,177.06,4.50,1.18,2024-02-29
4174980,INV-67799600062,02/29/2024,6129,EAST END LIQUOR / DES MOINES,3804 HUBBELL AVE,DES MOINES,50317,POINT (-93.541625959 41.631082004),NaN,POLK,...,CIROC PINEAPPLE,12,750,16.49,24.74,3,74.22,2.25,0.59,2024-02-29
4174977,INV-67800300005,02/29/2024,2201,HAPPY'S WINE & SPIRITS WHOLESALE,5925 UNIVERSITY AVE,CEDAR FALLS,50613,POINT (-92.429482021 42.512313981),NaN,BLACK HAWK,...,BULLEIT RYE 12 YR,12,750,27.49,41.24,1,41.24,0.75,0.19,2024-02-29
4174976,INV-67802400014,02/29/2024,10280,QUICK SHOP LIQUOR AND VAPE / CLEAR LAKE,904 NORTH 8TH STREET,CLEARLAKE,50428,POINT (-93.37843198 43.142923994),NaN,CERRO GORDO,...,EMPRESS 1908 GIN,6,1000,24.33,36.50,2,73.00,2.00,0.52,2024-02-29
4174973,INV-67800200079,02/29/2024,4988,HAPPY'S WINE & SPIRITS,5925 UNIVERSITY AVE,CEDAR FALLS,50613,POINT (-92.429482021 42.512313981),NaN,BLACK HAWK,...,GOLDSCHLAGER CINNAMON SCHNAPPS,12,750,13.50,20.25,2,40.50,1.50,0.39,2024-02-29
4174411,INV-67797700004,02/29/2024,5982,CASEY'S GENERAL STORE #6 / ALTOONA,407 8TH ST SW,ALTOONA,50009,POINT (-93.468928036 41.644324992),NaN,POLK,...,CROWN ROYAL REGAL APPLE MINI,10,50,5.91,8.87,20,298.60,1.00,0.26,2024-02-29


In [6]:
df = df[df['date_datetime'] < '2023-12']

In [7]:
print(df['date_datetime'].dt.year.unique())


[2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023]


In [8]:
# We save the file 2012-2023
# first we drop the datetime column for cleaninless
df = df.drop('date_datetime', axis=1)
df.to_csv('Iowa_Liquor_Sales_DEC2023.csv', index=False)  

In [9]:
import session_info
session_info.show(html=False)

-----
folium              0.14.0
geopandas           0.14.1
matplotlib          3.7.2
pandas              2.0.3
plotly              5.9.0
session_info        1.0.0
thefuzz             0.19.0
-----
IPython             8.15.0
jupyter_client      8.1.0
jupyter_core        5.3.0
-----
Python 3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:15:57) [MSC v.1916 64 bit (AMD64)]
Windows-10-10.0.22631-SP0
-----
Session information updated at 2024-03-19 13:18
